<a href="https://colab.research.google.com/github/vietai/back_translate/blob/master/colabs/Vietnamese_Backtranslation_Model_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Augmentation for Vietnamese Language by Backtranslation

Author: Thang Le

In this tutorial, we will experiment with using our trained Transformer models to translate interactively from Vietnamese to English and vice versa. We visualize attention weights in the translation models to help you get some idea how Transformer works. Lastly, we make use of these two translation models to do back-translation. Back-translation for languages with limited labeled data such as Vietnamese. The rest of this tutorial is organized as follows:
> 1. Mount to [Goolge Cloud Storage](https://cloud.google.com/storage/) for accessing our trained models
> 2. Clone some source codes needed
> 3. Prepare [tensor2tenor](https://github.com/tensorflow/tensor2tensor) models for inference
> 4. Interactive Translation
> 5. Attention Visualization
> 6. Vietnamese Data Augmentation by Back Translation

## I. Mount to Google Cloud Storage

We haved managed to train  **Vi --> En** and **En --> Vi** translation models and placed them on Google Cloud Storage. For inference purpose, we need to install **gcsfuse** to access our translation models on Google Cloud Storage

NOTE: In case you want to train you own translation models, here is the [colab](https://colab.research.google.com/drive/1Bx5HfxbmXnMK7kBLHlmGyhVhQVVrDI0p#scrollTo=cTUSADz_ti63) to check out! 

In [0]:
print('\nInstalling gcsfuse')
!echo "deb http://packages.cloud.google.com/apt gcsfuse-bionic main" > /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -
!apt -qq update
!apt -qq install gcsfuse


Installing gcsfuse
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   659  100   659    0     0  21258      0 --:--:-- --:--:-- --:--:-- 21966
OK
25 packages can be upgraded. Run 'apt list --upgradable' to see them.
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  gcsfuse
0 upgraded, 1 newly installed, 0 to remove and 25 not upgraded.
Need to get 4,274 kB of archives.
After this operation, 12.8 MB of additional disk space will be used.
Selecting previously unselected package gcsfuse.
(Reading database ... 131289 files and directories currently installed.)
Preparing to unpack .../gcsfuse_0.28.1_amd64.deb ...
Unpacking gcsfuse (0.28.1) ...
Setting up gcsfuse (0.28.1) ...


Let's import some necessary dependencies

In [0]:
from google.colab import auth
from itertools import product

import collections
import copy
import json
import os
import string
import re
import pprint

import numpy as np
import tensorflow as tf

# Enable TF Eager execution
tfe = tf.contrib.eager
tfe.enable_eager_execution()

auth.authenticate_user()

W0803 08:48:21.182517 140300811634560 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [0]:
# Now we mount the local storage to the google cloud bucket.
bucket = 'vien-translation'
print('Mounting bucket {} to local.'.format(bucket))
mount_point = '/content/{}'.format(bucket)

if not os.path.exists(mount_point):
  tf.gfile.MakeDirs(mount_point)

!fusermount -u $mount_point
!gcsfuse --implicit-dirs $bucket $mount_point
!ls $mount_point

Mounting bucket vien-translation to local.
fusermount: failed to unmount /content/vien-translation: Invalid argument
Using mount point: /content/vien-translation
Opening GCS connection...
Opening bucket...
Mounting file system...
File system has been successfully mounted.
checkpoints  data  foody_sa  raw  tmp


## II. Pull or Clone Some Source Codes

To do Vietnamese <--> English translation, we need to make use of Tensor2Tensor problems

### From our VietAI's github **vietai/back_translate**

We clone from vietai/back_translate for declaring ViEn problem

In [0]:
src = '/content/back_translate'
if not os.path.exists(src):
    !git clone https://github.com/vietai/back_translate.git
    %cd "back_translate"
else:
    %cd "back_translate"
    !git pull
!ls

Cloning into 'back_translate'...
remote: Enumerating objects: 76, done.
remote: Counting objects: 100% (76/76), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 76 (delta 43), reused 53 (delta 23), pack-reused 0
Unpacking objects: 100% (76/76), done.
/content/back_translate
back_translate.py  LICENSE	README.md	t2t_decoder.py
decoding.py	   problems.py	t2t_datagen.py	t2t_trainer.py


### From Tensor2Tensor
We then build Tensor2Tensor from source and import it for later usage. Because the tensor2tensor library provide EnVi problem only, we need to run **back_translate/problems.py** to register ViEn problem (for translating from Vietnamese in English) to tensor2tensor registry.

In [0]:
if not os.path.exists("tensor2tensor"):
    !git clone https://github.com/azraelzhor/tensor2tensor.git
    %cd tensor2tensor
else:
    %cd tensor2tensor
    !git pull

!pip install -q -v .

from tensor2tensor import models
from tensor2tensor import problems
from tensor2tensor.layers import common_layers
from tensor2tensor.utils import trainer_lib
from tensor2tensor.utils import t2t_model
from tensor2tensor.utils import registry
from tensor2tensor.utils import metrics
from tensor2tensor.visualization import attention
from tensor2tensor.data_generators import text_encoder, translate_envi
%run ../problems.py
%cd ..

Cloning into 'tensor2tensor'...
remote: Enumerating objects: 28133, done.
remote: Total 28133 (delta 0), reused 0 (delta 0), pack-reused 28133
Receiving objects: 100% (28133/28133), 13.27 MiB | 5.07 MiB/s, done.
Resolving deltas: 100% (22712/22712), done.
/content/back_translate/tensor2tensor
Processing /content/back_translate/tensor2tensor
     |████████████████████████████████| 143kB 40.5MB/s 
     |████████████████████████████████| 655kB 47.9MB/s 
     |████████████████████████████████| 942kB 51.3MB/s 
  Created wheel for tensor2tensor: filename=tensor2tensor-1.13.4-cp27-none-any.whl size=1539821 sha256=67c6094042a882f0c44b98f6fe1de74e06f5b046291f8e094ef1d4d9f6640f16
  Stored in directory: /tmp/pip-ephem-wheel-cache-SN03KR/wheels/9b/66/4b/c8d911abdc4eac28bab71c8ebbcf828ad59ab2cf1ad4aa5e12
  Created wheel for pypng: filename=pypng-0.0.20-cp27-none-any.whl size=67162 sha256=a89bccaf629dd7d9249aaff1a41a9265d27cf0730dc7c280c51711410d4718f7
  Stored in directory: /root/.cache/pip/wheels/

W0803 08:49:45.299123 140300811634560 deprecation_wrapper.py:119] From tensor2tensor/utils/expert_utils.py:68: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.

W0803 08:49:47.754268 140300811634560 deprecation_wrapper.py:119] From tensor2tensor/utils/adafactor.py:27: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0803 08:49:47.756233 140300811634560 deprecation_wrapper.py:119] From tensor2tensor/utils/multistep_optimizer.py:32: The name tf.train.AdamOptimizer is deprecated. Please use tf.compat.v1.train.AdamOptimizer instead.

W0803 08:49:47.814990 140300811634560 deprecation_wrapper.py:119] From /usr/local/lib/python2.7/dist-packages/mesh_tensorflow/ops.py:4237: The name tf.train.CheckpointSaverListener is deprecated. Please use tf.estimator.CheckpointSaverListener instead.

W0803 08:49:47.815905 140300811634560 deprecation_wrapper.py:119] From /usr/local/lib/python2.7/dist-packages/mesh_tensorflo

/content/back_translate


tensor2tensor/data_generators/video_generated.py:34: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/usr/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python2.7/dist-packages/traitlets/config/application.py", line 657, in launch_instance
    app.initialize(argv)
  File "</usr/local/lib/python2.7/dist-packages/decorator.pyc:decorator-gen-121>", line 2, in initialize
  File "/usr/local/lib/python2.7/dist-packages/traitlets/

## III. Prepare Tensor2Tensor Models for Inference

### General configuration
Here we set up some configuration to make sure that we get access to the right directories which store our trained translation models

In [0]:
envi_dir = os.path.join(mount_point, "checkpoints/translate_envi_iwslt32k_tiny/avg")
vien_dir = os.path.join(mount_point, "checkpoints/translate_vien_iwslt32k_tiny/avg")

envi_data_dir = os.path.join(mount_point, "data/translate_envi_iwslt32k")
vien_data_dir = os.path.join(mount_point, "data/translate_vien_iwslt32k")

vien_ckpt_path = os.path.join(vien_dir, "model.ckpt-50000")
envi_ckpt_path = os.path.join(envi_dir, "model.ckpt-50000")

Modes = tf.estimator.ModeKeys

# Create hparams and the model
model_name = "transformer"
hparams_set = "transformer_tiny"

### Vi_En Problem

In [0]:
vien_problem = problems.problem("translate_vien_iwslt32k")

# Get the encoders from the problem
vien_encoders = vien_problem.feature_encoders(vien_data_dir)


# Setup helper functions for encoding and decoding
def vien_encode(input_str, output_str=None):
  """Input str to features dict, ready for inference"""
  inputs = vien_encoders["inputs"].encode(input_str) + [1]  # add EOS id
  batch_inputs = tf.reshape(inputs, [1, -1, 1])  # Make it 3D.
  return {"inputs": batch_inputs, "target_space_id": tf.constant(1, dtype=tf.int32)}

def vien_decode(integers):
  """List of ints to str"""
  integers = list(np.squeeze(integers))
  if 1 in integers:
    integers = integers[:integers.index(1)]
  return vien_encoders["inputs"].decode(np.squeeze(integers))


hparams = trainer_lib.create_hparams(hparams_set, data_dir=vien_data_dir, problem_name="translate_vien_iwslt32k")

# NOTE: Only create the model once when restoring from a checkpoint; it's a
# Layer and so subsequent instantiations will have different variable scopes
# that will not match the checkpoint.
translate_vien_model = registry.model(model_name)(hparams, Modes.EVAL)


# Restore and translate!
def translate_vien(inputs, beam_size=4, alpha=0.6):
  encoded_inputs = vien_encode(inputs)

  with tfe.restore_variables_on_create(vien_ckpt_path):
    translated_outputs = translate_vien_model.infer(encoded_inputs, beam_size=beam_size, alpha=alpha)
        
  return vien_decode(translated_outputs["outputs"]), translated_outputs["cache"]

W0803 08:50:05.810746 140300811634560 deprecation_wrapper.py:119] From tensor2tensor/data_generators/text_encoder.py:938: The name tf.gfile.Exists is deprecated. Please use tf.io.gfile.exists instead.

W0803 08:50:06.525468 140300811634560 deprecation_wrapper.py:119] From tensor2tensor/data_generators/text_encoder.py:940: The name tf.gfile.Open is deprecated. Please use tf.io.gfile.GFile instead.

W0803 08:50:08.947407 140300811634560 deprecation_wrapper.py:119] From tensor2tensor/layers/common_layers.py:94: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0803 08:50:10.205029 140300811634560 deprecation_wrapper.py:119] From tensor2tensor/utils/t2t_model.py:244: The name tf.summary.text is deprecated. Please use tf.compat.v1.summary.text instead.



### En_Vi Problem

In [0]:
envi_problem = problems.problem("translate_envi_iwslt32k")

# Get the encoders from the problem
envi_encoders = envi_problem.feature_encoders(envi_data_dir)

envi_hparams = trainer_lib.create_hparams(hparams_set, data_dir=envi_data_dir, problem_name="translate_envi_iwslt32k")
translate_envi_model = registry.model(model_name)(envi_hparams, Modes.EVAL)


# Setup helper functions for encoding and decoding
def envi_encode(input_str, output_str=None):
  """Input str to features dict, ready for inference"""
  inputs = envi_encoders["inputs"].encode(input_str) + [1]  # add EOS id
  batch_inputs = tf.reshape(inputs, [1, -1, 1])  # Make it 3D.
  return {"inputs": batch_inputs, "target_space_id": tf.constant(1, dtype=tf.int32)}

def envi_decode(integers):
  """List of ints to str"""
  integers = list(np.squeeze(integers))
  if 1 in integers:
    integers = integers[:integers.index(1)]
  return envi_encoders["inputs"].decode(np.squeeze(integers))



def translate_envi(inputs, beam_size=4, alpha=0.6):
    encoded_inputs = envi_encode(inputs)
    
    with tfe.restore_variables_on_create(envi_ckpt_path):
        translated_outputs = translate_envi_model.infer(encoded_inputs, beam_size=beam_size, alpha=alpha)
        
    return envi_decode(translated_outputs["outputs"]), translated_outputs["cache"]

## IV. Interactive Translation
In this section, you can test the quality of our trained translation models by input any sentence in Vietnamese or English. The models will then translate the input sentence in one language and print out the output sentence in the other language.

### From Vietnamese to English

In [0]:
beam_size = 4 #@param {type: "integer"}
alpha = 0.6 #@param {type: "number"}
# Tôi là một giáo viên giỏi
vi_input_sentence = "Tôi là một giáo viên giỏi" #@param {type:"raw"}
en_output_sentence, _ = translate_vien(vi_input_sentence, beam_size=beam_size, alpha=alpha)
en_output_sentence = en_output_sentence.replace('&apos;', '\'')
print("The input sentence is tranlated as: \n{}".format(en_output_sentence))

W0803 08:51:05.001951 140300811634560 deprecation_wrapper.py:119] From tensor2tensor/utils/t2t_model.py:2296: The name tf.get_variable_scope is deprecated. Please use tf.compat.v1.get_variable_scope instead.

W0803 08:51:05.011019 140300811634560 deprecation_wrapper.py:119] From tensor2tensor/layers/modalities.py:475: The name tf.get_variable is deprecated. Please use tf.compat.v1.get_variable instead.

W0803 08:51:05.142455 140300811634560 deprecation.py:323] From tensor2tensor/layers/common_attention.py:857: to_float (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.cast` instead.
W0803 08:51:05.193547 140300811634560 deprecation.py:323] From tensor2tensor/utils/expert_utils.py:621: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0803 08:51:05.199455 140300811634

The input sentence is tranlated as: 
I 'm a good teacher .


### From English to Vietnamese

In [0]:
beam_size = 4 #@param {type: "integer"}
alpha = 0.6 #@param {type: "number"}

# I am a good teacher
en_input_sentence = "I am a good teacher" #@param {type:"raw"}
vi_output_sentence, _ = translate_envi(en_input_sentence, beam_size=beam_size, alpha=alpha)
print("The input sentence is tranlated as: \n{}".format(vi_output_sentence))

The input sentence is tranlated as: 
Tôi là một giáo viên tốt .


## V. Attention Visualization
In this section, we will visualize Transformer's attention weights to help you get some insights about how Transformer encode and decode sentences

In [0]:
inputs = "Tôi là một thầy giáo giỏi"
outputs, cache = translate_vien(inputs, beam_size=1, alpha=0)

print("Inputs: %s" % inputs)
print("Outputs: %s" % outputs.replace('&apos;', '\''))

W0803 08:52:23.519270 140300811634560 deprecation.py:323] From tensor2tensor/models/transformer.py:1221: to_int64 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.cast` instead.


Inputs: Tôi là một thầy giáo giỏi
Outputs: I 'm a good teacher .


In [0]:
SIZE = 35

def encode_eval(input_str, output_str, encoders):
  inputs = tf.reshape(encoders["inputs"].encode(input_str) + [1], [1, -1, 1, 1])  # Make it 3D.
  outputs = tf.reshape(encoders["inputs"].encode(output_str) + [1], [1, -1, 1, 1])  # Make it 3D.
  return {"inputs": inputs, "targets": outputs}

def get_att_mats(translate_model, hparams):
  enc_atts = []
  dec_atts = []
  encdec_atts = []

  for i in range(hparams.num_hidden_layers):
    enc_att = translate_model.attention_weights[
      "transformer/body/encoder/layer_%i/self_attention/multihead_attention/dot_product_attention" % i][0]
    dec_att = translate_model.attention_weights[
      "transformer/body/decoder/layer_%i/self_attention/multihead_attention/dot_product_attention" % i][0]
    encdec_att = translate_model.attention_weights[
      "transformer/body/decoder/layer_%i/encdec_attention/multihead_attention/dot_product_attention" % i][0]
    
    enc_atts.append(resize(enc_att))
    dec_atts.append(resize(dec_att))
    encdec_atts.append(resize(encdec_att))

  return enc_atts, dec_atts, encdec_atts

def resize(np_mat):
  # Sum across tokens
  np_mat = np_mat[:, :SIZE, :SIZE]
  row_sums = np.sum(np_mat, axis=-1)
  # Normalize
  layer_mat = np_mat / row_sums[:, np.newaxis]
  lsh = layer_mat.shape
  # Add extra dim for viz code to work.
  layer_mat = np.reshape(layer_mat, (1, lsh[0], lsh[1], lsh[2]))
  return layer_mat

def to_tokens(ids, hparams):
  ids = np.squeeze(ids)
  subtokenizer = hparams.problem_hparams.vocabulary['targets']
  tokens = []
  for _id in ids:
    if _id == 0:
      tokens.append('<PAD>')
    elif _id == 1:
      tokens.append('<EOS>')
    elif _id == -1:
      tokens.append('<NULL>')
    else:
        tokens.append(subtokenizer._subtoken_id_to_subtoken_string(_id))
  return tokens

def call_html():
  import IPython
  display(IPython.core.display.HTML('''
        <script src="/static/components/requirejs/require.js"></script>
        <script>
          requirejs.config({
            paths: {
              base: '/static/base',
              "d3": "https://cdnjs.cloudflare.com/ajax/libs/d3/3.5.8/d3.min",
              jquery: '//ajax.googleapis.com/ajax/libs/jquery/2.0.0/jquery.min',
            },
          });
        </script>
        '''))

In [0]:
# Get normalized attention weights for each layer
enc_atts, dec_atts, encdec_atts = get_att_mats(translate_vien_model, hparams)

dec_atts_revised = [np.repeat(dec_att, dec_att.shape[-1], axis=2) for dec_att in dec_atts]

attention_history = cache["attention_history"]
encdec_atts_revised = [attention_history["layer_0"], attention_history["layer_1"]]


In [0]:
inp_text = to_tokens(vien_encoders["inputs"].encode(inputs), hparams)
out_text = to_tokens(vien_encoders["inputs"].encode(outputs), hparams)

call_html()

attention.show(inp_text, out_text, enc_atts, dec_atts_revised, encdec_atts_revised)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## VI. Vietnamese Data Augmentation by Backtranslation
In this section, we can input any Vietnamese sentence and get its parapharases in an interactive way. By changing the **beam size** and **length penalty ($\alpha$)** hyperparameters , we can generate multiple paraphrases for each input sentence.




Below is some fun examples that we tried, you can then try it yourself. Have some fun :v


---

* Example 1: Bạn tôi học rất giỏi nhưng bạn không thích học
    * Bạn biết đấy , bạn biết đấy , bạn biết đấy , nhưng bạn không thích học .
    * Bạn rất giỏi ở trường , nhưng bạn không thích học .
---
* Example 2: Tôi là ai, và đây là đâu
    * Tôi là người mà người dân chủ , và đây là nơi nào ?
    * Tôi là ai là người manize , và đây là nơi nào ?
    * Tôi là người có thể tự nhiên , và đó là nơi mà nó ở đâu , và đó là nơi mà nó ở đâu ?
    * Tôi là người mà người có thể tự nhiên , và đó là nơi này ở đâu , và đó là nơi nào ?
---
* Example 3:  VietAI là một tổ chức phi lợi nhuận với mục tiêu thúc đẩy sự phát triển của công nghệ AI tại Việt Nam
    * VietAI là một tổ chức phi lợi nhuận với sự phát triển của AI ở Việt Nam .
---

NOTE: We experimented to investigate the effectiveness of Vietnamese augmented data by backtranslation on a sentiment analysis task for Foody comments and it seems promising :3. For more information, please check the [colab](https://colab.research.google.com/drive/1_I0KvFlHFyBcTRT3Bfx9BGLJcIHGJNrG#scrollTo=7yvhttVKTkZu) here.


In [0]:
beam_size = 4 #@param {type: "integer"}
alpha = 0.6 #@param {type: "number"}
# Tôi từng ước mơ trở thành cầu thủ bóng đá
vi_input_sentence = "Tôi từng ước mơ trở thành cầu thủ bóng đá" #@param {type:"raw"}
print("Augmented data:")

en_output_sentence, _ = translate_vien(vi_input_sentence, beam_size=beam_size, alpha=alpha)
vi_output_sentence, _ = translate_envi(en_output_sentence, beam_size=beam_size, alpha=alpha)
print(vi_output_sentence)


Augmented data:
Tôi đã từng mơ ước là một người chơi bóng đá .


## Acknowledgements

This work is made possible by [VietAI](http://vietai.org/). Special thanks to [Thang Luong](http://thangluong.com), [ Trieu H. Trinh](https://thtrieu.github.io/) and Phat Hoang for collaborating and giving comments.

## References

1. Improving Neural Machine Translation Models with Monolingual Data - Sennrich et al. , 2016a  ([arxiv](https://arxiv.org/abs/1511.06709))
2. Understanding Back-Translation at Scale - Edunov, Sergey, et al., 2018 ([arxiv](https://arxiv.org/abs/1808.09381))
3. T2T translate vi<->en tiny tpu - Trieu H. Trinh ([colab](https://colab.research.google.com/drive/1Bx5HfxbmXnMK7kBLHlmGyhVhQVVrDI0p))
4. Sentiment Analysis + Back translation - Trieu H. Trinh ([colab](https://colab.research.google.com/drive/1_I0KvFlHFyBcTRT3Bfx9BGLJcIHGJNrG#scrollTo=7yvhttVKTkZu))
5. Tensor2Tensor Intro - Tensor2Tensor Team([colab](https://colab.research.google.com/github/tensorflow/tensor2tensor/blob/master/tensor2tensor/notebooks/hello_t2t.ipynb))
